### Crawler obtido do site https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7434436/

Um primeiro conjunto de dados públicos do Twitter brasileiro e notícias do COVID-19 em português
Tiago de Melo, e Carlos MS Figueiredo 

#### Resumo
Neste artigo de dados, fornecemos uma coleção de 3.925.366 tweets e 18.413 notícias online sobre a discussão online sobre COVID-19 no Brasil. Os dados do Twitter foram coletados por meio da biblioteca Twitterscraper Python e foi considerado um conjunto de palavras-chave em português referentes ao COVID-19. Para facilitar a identificação de tweets que possuem hashtags, mídia e retuítes para pesquisadores ou entusiastas de dados, criamos três conjuntos de dados específicos para cada uma dessas categorias. As notícias do COVID-19 foram coletadas no portal UOL, o site brasileiro mais popular. Todos os dados foram coletados de janeiro a maio de 2020. Esses conjuntos de dados podem atrair a atenção de comunidades como ciência de dados, ciências sociais, processamento de linguagem natural, turismo, infodemiologia e saúde pública.

Palavras-chave: COVID-19, Pandemia, Conjunto de dados, Twitter, Notícias, Português

In [2]:
import os
import requests
import sys

In [3]:
#Place where the files will be saved
dir_output = 'terapages'

In [4]:
# Procedure to create a directory to save the pages
try:
    os.mkdir(dir_output)
except OSError:
    print ("Directory %s already exists." % dir_output)
else:
    print ("Successfully created the directory %s " % dir_output)


Directory terapages already exists.


In [5]:
#List of URLs gathered manually
input_file = "terauol-link.txt"

In [6]:
terapages_collected = next(os.walk('terapages'))[2]
# terapages_collected = []

with open(input_file) as infile:
    for line in infile:
        url = line.strip()
        page_name = url.split('/')[-1]
        if page_name in terapages_collected:
            print ("Page already collected.")
#            print (page_name)
            continue
        elif page_name.split('.')[-1] != "htm":
            print ("Link is not a pageLink não é uma página HTML")
            continue
        print (url)
        terapages = requests.get(url)
        with open(dir_output + '/' + page_name, 'w') as f:
            f.write(terapages.text)


Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.
Page already collected.


### Criando o arquivo .csv e Lendo o Conteúdo

In [7]:
import glob
import os
from bs4 import BeautifulSoup
import codecs
import datetime
import json
import csv 

In [8]:
input_file = "terauol-link.txt"

In [9]:
input_file

'terauol-link.txt'

In [10]:
#Procedure to get the list of pages collected previously
d = {}
with open(input_file) as infile:
    for line in set(infile):
        url = line.strip()
        page_name = url.split('/')[-1]
        day = line.strip().split('/')[-2]
        month = line.strip().split('/')[-3]
        year = line.strip().split('/')[-4]
        try:
            data_page = datetime.datetime.strptime(day + '-' + month + '-' + year, '%d-%m-%Y')
        except:
            None
        if data_page.month > 5 or  data_page.year < 2020:
            None
        else:
            d[page_name] = [url, str(data_page)]

In [11]:
#Procedure to create a structure (d) with the data collected
dir_input = "terapages"
pages_collected = next(os.walk(dir_input))[2]

# pages_collected = []

for page in pages_collected:
    if not page in d.keys():
        continue
    text_uol = []
    soup = BeautifulSoup(open(dir_input + '/' + page), "html.parser")
    divTag = soup.find_all("div", {"class": "text"})
    for tag in divTag:
        tdTags = tag.find_all("p")
        for tag in tdTags:
            if len(tag.text) < 10:
                continue
            text_uol.append(tag.text)
    d[page].append(text_uol)

In [12]:
#Procedure to save the data into file terauol-dataframe.csv
header = ['date', 'title', 'url', 'text']
with open("terauol.csv", "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    content = []
    for k,v in d.items():
        if len(v) == 2:
            continue
        url = v[0]
        title = url.split('/')[-1].split('.')[0].replace('-', ' ')
        data_page = v[1]
        text_uol = v[2]
        line = data_page, title, url, text_uol
        content.append(line)
    # write the actual content line by line
    newL = sorted(content, key=lambda x: x[0], reverse=False)
    for l in newL:
        writer.writerow(l)

### Lendo o arquivo .csv e Imprimindo uma saída para verificação

In [13]:
# para ver conteudo das variaveis
import pandas as pd

In [14]:
file_name = "terauol.csv"
df = pd.read_csv(file_name, encoding="ISO-8859-1", usecols=["date", "title", "url", "text"])

In [15]:
df

,date,title,url,text
0,2021-03-11 00:00:00,remedios contra covid 19,https://www.uol.com.br/vivabem/noticias/redaca...,"['Após um ano da pandemia do novo coronavírus,..."
1,2021-03-20 00:00:00,covid 19 dificuldade de concentracao e memoria...,https://www.uol.com.br/vivabem/noticias/bbc/20...,['Sintomas podem afetar as pessoas por meses a...
2,2021-03-23 00:00:00,coronavirus permanece ativo por mais de 14 dia...,https://www.uol.com.br/vivabem/noticias/redaca...,['Estudos conduzidos no Instituto de Medicina ...
3,2021-03-23 00:00:00,tratamento precoce covid cloroquina nao funciona,https://noticias.uol.com.br/confere/ultimas-no...,"['Há um ano, se luta contra a pandemia gerada ..."
4,2021-03-28 00:00:00,tecnologia deveria ser aberta para qualquer pa...,https://www.uol.com.br/vivabem/noticias/agenci...,['Diante do risco de países pobres não consegu...
5,2021-03-28 00:00:00,efeitos da covid 21 dos infectados relataram s...,https://www.uol.com.br/vivabem/noticias/bbc/20...,['Lesão na pele pode servir tanto como um sina...
6,2021-03-28 00:00:00,covid 19 trombose pode acontecer ate quatro se...,https://www.uol.com.br/vivabem/noticias/redaca...,['Uma pesquisa da Sociedade Brasileira de Angi...
7,2021-03-30 00:00:00,4 boas noticias sobre novos tratamentos em tes...,https://www.uol.com.br/vivabem/noticias/bbc/20...,['Corrida por medicamentos já levou ao registr...
8,2021-03-31 00:00:00,como funciona a vacina da johnson aprovada par...,https://www.uol.com.br/vivabem/noticias/bbc/20...,['A Agência Nacional de Vigilância Sanitária (...
9,2021-03-31 00:00:00,vivabem lanca campanha para conscientizar sobr...,https://economia.uol.com.br/noticias/redacao/2...,['Vacinas funcionam e são seguras. Para inspir...
